<a href="https://colab.research.google.com/github/nitiksha/PySpark_code_practice/blob/main/lead_lag.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from pyspark.sql import SparkSession, Window
import pyspark.sql.functions as F

# Sample data
data = [
    (1, "2015-01-01", 10),
    (2, "2015-01-02", 25),
    (3, "2015-01-03", 20),
    (4, "2015-01-04", 30),
]
columns = ["id", "recordDate", "temperature"]

# Create DataFrame
spark = SparkSession.builder.getOrCreate()
df = spark.createDataFrame(data, columns)

In [12]:
window_spec = Window.orderBy(F.col('recordDate').asc())

In [13]:
df = df.withColumn("lag_date",F.lag('recordDate',1).over(window_spec))\
          .withColumn("lead_temp",F.lead('temperature',1).over(window_spec))

In [14]:
df.show()

+---+----------+-----------+----------+---------+
| id|recordDate|temperature|  lag_date|lead_temp|
+---+----------+-----------+----------+---------+
|  1|2015-01-01|         10|      NULL|       25|
|  2|2015-01-02|         25|2015-01-01|       20|
|  3|2015-01-03|         20|2015-01-02|       30|
|  4|2015-01-04|         30|2015-01-03|     NULL|
+---+----------+-----------+----------+---------+



In [23]:
df = df.filter((F.datediff(F.col('recordDate'),F.col('lag_date')) == 1)  & (F.col('lead_temp')>F.col('temperature')))

In [24]:
df.show()

+---+----------+-----------+----------+---------+
| id|recordDate|temperature|  lag_date|lead_temp|
+---+----------+-----------+----------+---------+
|  3|2015-01-03|         20|2015-01-02|       30|
+---+----------+-----------+----------+---------+

